In [ ]:
import os
import csv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib widget

In [ ]:
sns.set_style("whitegrid")
sns.set_context("talk")
sns.set_palette("pastel")
fig_height = 5

In [ ]:
# plots mean AUC +/- std for each model
# Inputs: model names as a list, Pandas DataFrame of exported results CSV from Google Spreadsheets

def plot_performance(
    model_names: list,
    df: pd.DataFrame,
    y_min: float = 0.7,
    y_max: float = 0.95,
    x_label_strs: list = [],
    plot_title: str = None):

    y_delta = 0.05
    
    stats = {model: {'aucs': [], 'mean': 0, 'std': 0} for model in model_names}

    for model in model_names:
        aucs = []
        for bootstrap in range(10):
            auc_str = df[df.index == str(bootstrap)][model].values[0]
            aucs.append(float(auc_str))

        stats[model]['aucs'] = aucs    
        stats[model]['mean'] = np.mean(aucs)
        stats[model]['std'] = np.std(aucs)
        
    means = [stats[model]['mean'] for model in model_names]
    stds = [stats[model]['std'] for model in model_names]
        
    fig_width = len(model_names) * 1.25
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))
    x_pos = np.arange(len(model_names))
    
    bar = sns.barplot(x=x_pos, y=means, yerr=stds)
    ax.set_ylabel('AUC')
    ax.set_ylim([y_min, y_max])
    ax.set_yticks(np.arange(y_min, y_max+0.01, y_delta))
    
    ax.set_xticks(range(len(model_names)))
    ax.set_xticklabels(model_names if len(x_label_strs) == 0 else x_label_strs)

    if plot_title is not None:
        ax.set_title(plot_title)
    
    offset = -0.25
    for i, v in enumerate(means):
        plt.text(x_pos[i]+offset, y_min + 0.01, f'{v:0.2f}')
    plt.xticks(rotation=90)
    plt.tight_layout()

In [ ]:
def get_aucs_from_column(df: pd.DataFrame, col_name: str, bootstraps: int = 10) -> list:
    """Given a dataframe and model name (col_name),
    extracts the AUCs and casts to list of floats"""
    aucs = []
    if col_name not in df:
        return aucs
    for bootstrap in range(bootstraps):
        try:
            auc = df[col_name].loc[str(bootstrap)]
            aucs.append(float(auc))
        except:
            aucs.append(np.nan)
            print(f"no valid auc found at bootstrap {bootstrap}")
    return aucs

## Load CSVs and concatenate horizontally into one wide dataframe

In [ ]:
csv_filenames = ['stsnet', 'shallow', 'sts-cabg', 'sts-valve', 'sts-cabg-valve']
#csv_filenames = ['stsnet', 'sts-cabg']
rootdir = os.path.expanduser("~/dropbox/sts-ecg/figures-and-tables")
df = pd.DataFrame()
for csv_filename in csv_filenames:
    fpath = os.path.join(rootdir, f"{csv_filename}.csv")
    df_ = pd.read_csv(fpath, low_memory=False, index_col=0)
    if np.any(["sts" not in col for col in df_.columns]):
        cols = [f"{csv_filename}_{col}" for col in df_.columns]
        df_.columns = cols
    df = pd.concat([df, df_], axis=1)

df.rename(columns={'Unnamed: 0':'parameter'}, inplace=True )

## Get AUCs from dataframe as a dict (keyed by model name) of lists of floats

In [ ]:
aucs = {}
for model in df.keys():
    aucs[model] = get_aucs_from_column(df=df, col_name=model, bootstraps=10)

In [ ]:
outcomes = [
    "death",
    "stroke",
    "renal",
    "vent",
    "reop",
    "stay",
    "dsw",
]

for outcome in outcomes:
    models_to_plot = [
        f'sts-cabg_{outcome}',
        f'sts-valve_{outcome}',
        f'sts-cabg-valve_{outcome}',
        f'sts-shallow-v001-{outcome}',
        f'stsnet-v057-{outcome}',
        f'stsnet-v046 ({outcome})'
    ]

    model_1 = f'sts-shallow-v001-{outcome}'
    model_2 = f'stsnet-v057-{outcome}'
    statistic, p_val = stats.ttest_rel(aucs[model_1], aucs[model_2])
    print(f"mean AUC = {np.mean(aucs[model_1]):0.2f} ± {np.std(aucs[model_1]):0.2f} from model {model_1} ")
    print(f"mean AUC = {np.mean(aucs[model_2]):0.2f} ± {np.std(aucs[model_2]):0.2f} from model {model_2} ")
    print(f"p-value = {p_val:0.2f}")
    p_text_1 = "*" if p_val < 0.05 else ""

    model_1 = f'sts-shallow-v001-{outcome}'
    model_2 = f'stsnet-v046 ({outcome})'
    statistic, p_val = stats.ttest_rel(aucs[model_1], aucs[model_2])
    print(f"mean AUC = {np.mean(aucs[model_1]):0.2f} ± {np.std(aucs[model_1]):0.2f} from model {model_1} ")
    print(f"mean AUC = {np.mean(aucs[model_2]):0.2f} ± {np.std(aucs[model_2]):0.2f} from model {model_2} ")
    print(f"p-value = {p_val:0.2f}")
    p_text_2 = "*" if p_val < 0.05 else ""

    plot_performance(
        model_names=models_to_plot,
        df=df,
        y_min=0.5,
        y_max=0.9,
        x_label_strs=[
            "STS CABG",
            "STS Valve",
            "STS CABG+VALVE",
            "LogReg",
            f"STSNet (single task){p_text_1}",
            f"STSNet (multitask){p_text_2}"
        ],
        plot_title=outcome,
    )